<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [8]:
import pandas as pd

# Minimal dataset with LaunchSite, Latitude, Longitude, and dummy Class
data = {
    'LaunchSite': [
        'CCAFS LC-40',
        'CCAFS SLC-40',
        'KSC LC-39A',
        'VAFB SLC-4E'
    ],
    'Latitude': [
        28.5618571,   # Cape Canaveral LC-40
        28.562302,    # Cape Canaveral SLC-40
        28.573255,    # Kennedy Space Center LC-39A
        34.632834     # Vandenberg AFB SLC-4E
    ],
    'Longitude': [
        -80.577366,   # LC-40
        -80.577356,   # SLC-40
        -80.646895,   # LC-39A
        -120.610746   # SLC-4E
    ],
    'Class': [1, 1, 0, 1]  # dummy success/failure flag
}

df = pd.DataFrame(data)
df.head()

,LaunchSite,Latitude,Longitude,Class
0,CCAFS LC-40,28.561857,-80.577366,1
1,CCAFS SLC-40,28.562302,-80.577356,1
2,KSC LC-39A,28.573255,-80.646895,0
3,VAFB SLC-4E,34.632834,-120.610746,1


In [9]:
# Reuse df from the previous lab
# If you restarted the kernel, reload the cleaned dataset you built earlier
df.head()

,LaunchSite,Latitude,Longitude,Class
0,CCAFS LC-40,28.561857,-80.577366,1
1,CCAFS SLC-40,28.562302,-80.577356,1
2,KSC LC-39A,28.573255,-80.646895,0
3,VAFB SLC-4E,34.632834,-120.610746,1


In [10]:
## Task 1: Mark all launch sites on a map
import folium
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon

# Grab the unique sites with their representative (mean) coordinates
sites = (
    df.groupby('LaunchSite', as_index=False)[['Latitude','Longitude']].mean()
    .rename(columns={'Latitude':'lat','Longitude':'lon'})
)

# Center the map roughly on the mean of all points
m = folium.Map(location=[sites.lat.mean(), sites.lon.mean()], zoom_start=6, tiles='cartodbpositron')

# Mouse readout (lat/lon)
MousePosition().add_to(m)

# Cluster for site pins
site_cluster = MarkerCluster(name='Launch Sites').add_to(m)

# Add a marker for each unique site
for _, r in sites.iterrows():
    folium.Marker(
        location=[r.lat, r.lon],
        popup=folium.Popup(f"<b>{r.LaunchSite}</b>", max_width=250),
        tooltip=r.LaunchSite,
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(site_cluster)

# Label each site on the map (nice for screenshots)
for _, r in sites.iterrows():
    folium.map.Marker(
        [r.lat, r.lon],
        icon=DivIcon(icon_size=(150,36),
                     icon_anchor=(0,0),
                     html=f'<div style="font-size:12px; font-weight:600;">{r.LaunchSite}</div>')
    ).add_to(m)

m

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [11]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [12]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [13]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [14]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [15]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.

Equator? No. None of the SpaceX U.S. launch sites are near the equator—they sit around 28.5° N (CCAFS/KSC in Florida) and 34.6° N (Vandenberg in California). They’re low latitude, which helps a bit for eastward launches, but not equatorial.

Coast? Yes. All of them are right on the coast (Atlantic for CCAFS/KSC, Pacific for Vandenberg).

In [16]:
# Task 2: Mark the success/failed launches for each site on the map
spacex_map = folium.Map(location=[launch_sites_df.Lat.mean(), launch_sites_df.Long.mean()],
                        zoom_start=5, tiles="cartodbpositron")

# Iterate through each row in the dataframe
for _, row in spacex_df.iterrows():
    # Select color based on class (success/failure)
    color = 'green' if row['class'] == 1 else 'red'
    
    # Add a CircleMarker for each launch
    folium.CircleMarker(
        location=[row['Lat'], row['Long']],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"Site: {row['Launch Site']}<br>Outcome: {'Success' if row['class'] == 1 else 'Failure'}"
    ).add_to(spacex_map)

# Show the map
spacex_map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [17]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [19]:
marker_cluster = MarkerCluster()
# Base map centered on mean coordinates
spacex_map = folium.Map(
    location=[launch_sites_df.Lat.mean(), launch_sites_df.Long.mean()],
    zoom_start=5, tiles="cartodbpositron"
)

# Marker cluster for individual launches
marker_cluster = MarkerCluster(name='Launches').add_to(spacex_map)

In [20]:
def outcome_color(c):  # class: 1=success, 0=failure
    return 'green' if int(c) == 1 else 'red'

for _, r in spacex_df.dropna(subset=['Lat','Long']).iterrows():
    folium.CircleMarker(
        location=[r['Lat'], r['Long']],
        radius=5,
        color=outcome_color(r['class']),
        fill=True,
        fill_color=outcome_color(r['class']),
        fill_opacity=0.8,
        popup=(
            f"<b>Site:</b> {r['Launch Site']}<br>"
            f"<b>Outcome:</b> {'Success' if r['class']==1 else 'Failure'}"
        ),
    ).add_to(marker_cluster)

In [21]:
# Compute per-site success rate
site_stats = (
    spacex_df.groupby('Launch Site', as_index=False)
    .agg(success_rate=('class','mean'),
         successes=('class','sum'),
         launches=('class','count'))
    .merge(launch_sites_df, on='Launch Site', how='left')
)

# Add site pins with popup stats
for _, s in site_stats.iterrows():
    folium.Marker(
        [s.Lat, s.Long],
        tooltip=s['Launch Site'],
        popup=(f"<b>{s['Launch Site']}</b><br>"
               f"Success rate: {s.success_rate:.0%}<br>"
               f"Successes: {int(s.successes)} / Launches: {int(s.launches)}"),
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(spacex_map)

folium.LayerControl(collapsed=False).add_to(spacex_map)
spacex_map

*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [23]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

# Create a new column marker_color based on the class value
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Quick check
spacex_df[['Launch Site', 'class', 'marker_color']].head(10)

,Launch Site,class,marker_color
0,CCAFS LC-40,0,red
1,CCAFS LC-40,0,red
2,CCAFS LC-40,0,red
3,CCAFS LC-40,0,red
4,CCAFS LC-40,0,red
5,CCAFS LC-40,0,red
6,CCAFS LC-40,0,red
7,CCAFS LC-40,0,red
8,CCAFS LC-40,0,red
9,CCAFS LC-40,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [27]:
import folium
from folium.plugins import MarkerCluster

# 1) Base map (USA view)
usa_map = folium.Map(
    location=[launch_sites_df.Lat.mean(), launch_sites_df.Long.mean()],
    zoom_start=4, tiles="cartodbpositron"
)

# 2) Cluster of all launches (green=success, red=failure)
cluster = MarkerCluster(name='Launches').add_to(usa_map)

for _, r in spacex_df.dropna(subset=['Lat','Long']).iterrows():
    color = 'green' if int(r['class']) == 1 else 'red'
    folium.CircleMarker(
        location=[r['Lat'], r['Long']],
        radius=5, color=color, fill=True, fill_color=color, fill_opacity=0.8,
        popup=f"{r['Launch Site']} | {'Success' if r['class']==1 else 'Failure'}"
    ).add_to(cluster)

# 3) Pin each site so it’s easy to spot the locations
for _, s in launch_sites_df.iterrows():
    folium.Marker(
        [s.Lat, s.Long], tooltip=s['Launch Site'],
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(usa_map)

usa_map

In [28]:
# 1) Pick a site row (here: CCAFS SLC-40)
target = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'CCAFS SLC-40'].iloc[0]
center = [target.Lat, target.Long]

# 2) Close-up map
site_map = folium.Map(location=center, zoom_start=17, tiles='OpenStreetMap')

# 3) Draw two orange radius circles (meters)
folium.Circle(center, radius=1000, color='orange', fill=True, fill_opacity=0.10).add_to(site_map)
folium.Circle(center, radius=500,  color='orange', fill=True, fill_opacity=0.10).add_to(site_map)

# 4) Plot the individual launches that occurred at this site
for _, r in spacex_df[spacex_df['Launch Site'] == 'CCAFS SLC-40'].iterrows():
    color = 'green' if int(r['class']) == 1 else 'red'
    folium.CircleMarker(
        location=[r['Lat'], r['Long']],
        radius=6, color=color, fill=True, fill_color=color, fill_opacity=0.9
    ).add_to(site_map)

# 5) Optional label pin
folium.Marker(center, tooltip='CCAFS SLC-40').add_to(site_map)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [ ]:
# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [29]:
site_row = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'CCAFS SLC-40'].iloc[0]
site_lat, site_lon = float(site_row.Lat), float(site_row.Long)

site_map = folium.Map(location=[site_lat, site_lon], zoom_start=16, tiles='cartodbpositron')
site_map

In [31]:
proximities = {
    'Coastline':  (28.41000, -80.56500),
    'Railway':    (28.52000, -80.80000),
    'Highway':    (28.49500, -80.78000),
    'CityCenter': (28.53800, -81.37900)
}

In [32]:
import math, pandas as pd

def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0088
    φ1, φ2 = math.radians(lat1), math.radians(lat2)
    dφ = math.radians(lat2 - lat1)
    dλ = math.radians(lon2 - lon1)
    a = math.sin(dφ/2)**2 + math.cos(φ1)*math.cos(φ2)*math.sin(dλ/2)**2
    return 2 * R * math.asin(math.sqrt(a))

rows = []
for name, (plat, plon) in proximities.items():
    dist = round(haversine_km(site_lat, site_lon, plat, plon), 2)
    rows.append({'Launch Site': site_row['Launch Site'], 'Proximity': name, 'Distance_km': dist})
dist_df = pd.DataFrame(rows)
dist_df

,Launch Site,Proximity,Distance_km
0,CCAFS SLC-40,Coastline,17.07
1,CCAFS SLC-40,Railway,22.32
2,CCAFS SLC-40,Highway,21.25
3,CCAFS SLC-40,CityCenter,78.40


In [33]:
folium.Marker([site_lat, site_lon],
              tooltip=site_row['Launch Site'],
              icon=folium.Icon(color='blue', icon='rocket', prefix='fa')).add_to(site_map)

folium.Circle([site_lat, site_lon], radius=500,  color='orange', fill=True, fill_opacity=0.08).add_to(site_map)
folium.Circle([site_lat, site_lon], radius=1000, color='orange', fill=True, fill_opacity=0.08).add_to(site_map)

for name, (plat, plon) in proximities.items():
    dkm = haversine_km(site_lat, site_lon, plat, plon)
    folium.Marker([plat, plon],
                  tooltip=name,
                  popup=f"{name}<br>{dkm:.2f} km from {site_row['Launch Site']}",
                  icon=folium.Icon(color='gray', icon='flag', prefix='fa')).add_to(site_map)
    folium.PolyLine([[site_lat, site_lon], [plat, plon]], color='purple', weight=2, opacity=0.7).add_to(site_map)

site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [36]:
# choose one site (change name as needed)
site_name = 'CCAFS SLC-40'  # or 'KSC LC-39A', 'CCAFS LC-40', 'VAFB SLC-4E'
site_row = launch_sites_df.loc[launch_sites_df['Launch Site'] == site_name].iloc[0]
site_lat, site_lon = float(site_row.Lat), float(site_row.Long)
site_lat, site_lon

(28.56319718, -80.57682003)

In [37]:
# replace these with the coordinates you read on the map
proximities = {
    'Coastline':  (28.41000, -80.56500),
    'Railway':    (28.52000, -80.80000),
    'Highway':    (28.49500, -80.78000),
    'CityCenter': (28.53800, -81.37900),
}

In [38]:
import pandas as pd

rows = []
for name, (plat, plon) in proximities.items():
    d_km = calculate_distance(site_lat, site_lon, plat, plon)
    rows.append({'Launch Site': site_name, 'Proximity': name, 'Distance_km': round(d_km, 2)})

dist_df = pd.DataFrame(rows).sort_values('Distance_km')
dist_df

,Launch Site,Proximity,Distance_km
0,CCAFS SLC-40,Coastline,17.08
2,CCAFS SLC-40,Highway,21.26
1,CCAFS SLC-40,Railway,22.33
3,CCAFS SLC-40,CityCenter,78.43


In [39]:
# zoomed map on the chosen site
site_map = folium.Map(location=[site_lat, site_lon], zoom_start=16, tiles='cartodbpositron')

# site marker + 500 m and 1 km buffers
folium.Marker([site_lat, site_lon], tooltip=site_name,
              icon=folium.Icon(color='blue', icon='rocket', prefix='fa')).add_to(site_map)
folium.Circle([site_lat, site_lon], radius=500,  color='orange', fill=True, fill_opacity=0.08).add_to(site_map)
folium.Circle([site_lat, site_lon], radius=1000, color='orange', fill=True, fill_opacity=0.08).add_to(site_map)

# add proximity markers and lines with distance in popup
for name, (plat, plon) in proximities.items():
    d_km = calculate_distance(site_lat, site_lon, plat, plon)
    folium.Marker([plat, plon], tooltip=name,
                  popup=f"{name}<br>{d_km:.2f} km from {site_name}",
                  icon=folium.Icon(color='gray', icon='flag', prefix='fa')).add_to(site_map)
    folium.PolyLine([[site_lat, site_lon], [plat, plon]], color='purple', weight=2, opacity=0.7).add_to(site_map)

site_map

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [42]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
# --- Choose the launch site you centered earlier ---
# If you already have site_row/site_lat/site_lon, you can reuse them.
site_name = 'CCAFS SLC-40'  # change if needed
site_row = launch_sites_df.loc[launch_sites_df['Launch Site'] == site_name].iloc[0]
launch_site_lat, launch_site_lon = float(site_row.Lat), float(site_row.Long)

# --- Enter the coastline point you read from the map (MousePosition) ---
# Example from your comment: Lat: 28.56367, Lon: -80.57163  <-- replace with your exact numbers
coastline_lat  = 28.56367
coastline_lon  = -80.57163

# --- Compute distance in km ---
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"Distance from {site_name} to coastline: {distance_coastline:.2f} km")

# Add a marker for the coastline point and a line to the site (on site_map or spacex_map)
folium.Marker([coastline_lat, coastline_lon],
              tooltip='Coastline',
              icon=folium.Icon(color='gray', icon='flag', prefix='fa')).add_to(site_map)

folium.PolyLine([[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
                color='purple', weight=2, opacity=0.7).add_to(site_map)

site_map

Distance from CCAFS SLC-40 to coastline: 0.51 km


In [43]:
# --- Distance marker at midpoint between site and coastline ---
mid_lat = (launch_site_lat + coastline_lat) / 2
mid_lon = (launch_site_lon + coastline_lon) / 2

distance_marker = folium.Marker(
    [mid_lat, mid_lon],
    icon=DivIcon(
        icon_size=(120, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline),
    )
)

# Add to map
distance_marker.add_to(site_map)

# Also add a line and markers if not already drawn
folium.PolyLine([[launch_site_lat, launch_site_lon],
                 [coastline_lat, coastline_lon]],
                color='blue', weight=2, opacity=0.6).add_to(site_map)

folium.Marker([coastline_lat, coastline_lon],
              tooltip='Coastline',
              icon=folium.Icon(color='gray', icon='flag')).add_to(site_map)

site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [44]:
# Example: using CCAFS SLC-40 site
launch_lat, launch_lon = 28.563197, -80.576820     # launch site coordinate
coast_lat, coast_lon   = 28.56367, -80.57163       # closest coastline coordinate (you marked earlier)

# Create a PolyLine between the two coordinates
lines = folium.PolyLine(
    locations=[(launch_lat, launch_lon), (coast_lat, coast_lon)],
    weight=2, color='blue'
)

# Add the line to your site_map
site_map.add_child(lines)

site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [45]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
# Example launch site (CCAFS SLC-40)
launch_lat, launch_lon = 28.563197, -80.576820

# Example: closest city (Orlando area – replace with MousePosition coords you collected)
city_lat, city_lon = 28.53800, -81.37900  

# Calculate distance
distance_city = calculate_distance(launch_lat, launch_lon, city_lat, city_lon)

# Create a marker at the city with distance label
city_marker = folium.Marker(
    location=[city_lat, city_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_city),
    )
)
site_map.add_child(city_marker)

# Draw PolyLine from launch site to city
city_line = folium.PolyLine(
    locations=[(launch_lat, launch_lon), (city_lat, city_lon)],
    weight=2, color='green'
)
site_map.add_child(city_line)

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.

Railways: Yes—generally close. Launch sites (esp. CCAFS/KSC) sit a few km from rail lines used to move large components and propellants. Your measured rail distances should be small (typically single-digit km).

Highways: Yes—close. Each site has nearby highway access for routine logistics and workforce commuting. Distances are usually a few km.

Coastline: Yes—very close. All sites are coastal by design so rockets fly out over water and so ASDS (drone ship) recovery is feasible. Your coastline distances likely came out < 1–2 km.

Cities: Kept at a safe distance. Major population centers are tens of km away (e.g., Orlando is ~80+ km from KSC/CCAFS). This reduces public risk, meets range safety constraints, and minimizes overflight of populated areas.

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
